In [12]:
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()


'/Users/mac/.wdm/drivers/chromedriver/mac64/129.0.6668.89/chromedriver-mac-x64/chromedriver'

In [13]:
from selenium import webdriver
browser = webdriver.Chrome()


In [15]:
url = 'https://google.com'
browser.get(url)

In [16]:
from selenium.webdriver.common.by import By
browser.find_element(By.CLASS_NAME, 'uU7dJb').text

'대한민국'

In [17]:
url = 'https://www.yes24.com/Product/category/bestseller?CategoryNumber=001&sumgb=06'
browser.get(url)

In [18]:
# 1페이지 링크 데이터 전부 수집

### 한 개의 베스트 셀러 링크 데이터 수집
browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

### 1페이지 전체의 링크 데이터 
datas = browser.find_elements(By.CLASS_NAME, 'gd_name')

for i in datas :
    print(i.get_attribute('href'))

https://www.yes24.com/Product/Goods/133213071
https://www.yes24.com/Product/Goods/125557465
https://www.yes24.com/Product/Goods/131930939
https://www.yes24.com/Product/Goods/133312039
https://www.yes24.com/Product/Goods/133870359
https://www.yes24.com/Product/Goods/104663596
https://www.yes24.com/Product/Goods/133236068
https://www.yes24.com/Product/Goods/132560712
https://www.yes24.com/Product/Goods/133199014
https://www.yes24.com/Product/Goods/117014613
https://www.yes24.com/Product/Goods/133308510
https://www.yes24.com/Product/Goods/134331993
https://www.yes24.com/Product/Goods/128266166
https://www.yes24.com/Product/Goods/124999476
https://www.yes24.com/Product/Goods/124539805
https://www.yes24.com/Product/Goods/8759796
https://www.yes24.com/Product/Goods/129602366
https://www.yes24.com/Product/Goods/132565140
https://www.yes24.com/Product/Goods/133819718
https://www.yes24.com/Product/Goods/118322901
https://www.yes24.com/Product/Goods/133138701
https://www.yes24.com/Product/Goods/

In [19]:
# 3페이지 까지의 링크 데이터 전부 수집
import time

link_list = []
for i in range (1, 4) :
    print('*' * 10, f"현재 {i} 페이지 수집 중입니다.", "*" * 10)
    url = f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24'
    browser.get(url)

    browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

    datas = browser.find_elements(By.CLASS_NAME, 'gd_name')

    for i in datas : 
        link = i.get_attribute('href')
        link_list.append(link)

    time.sleep(3)

print(link_list)


********** 현재 1 페이지 수집 중입니다. **********
********** 현재 2 페이지 수집 중입니다. **********
********** 현재 3 페이지 수집 중입니다. **********
['https://www.yes24.com/Product/Goods/133213071', 'https://www.yes24.com/Product/Goods/125557465', 'https://www.yes24.com/Product/Goods/131930939', 'https://www.yes24.com/Product/Goods/133312039', 'https://www.yes24.com/Product/Goods/133870359', 'https://www.yes24.com/Product/Goods/104663596', 'https://www.yes24.com/Product/Goods/133236068', 'https://www.yes24.com/Product/Goods/132560712', 'https://www.yes24.com/Product/Goods/133199014', 'https://www.yes24.com/Product/Goods/117014613', 'https://www.yes24.com/Product/Goods/133308510', 'https://www.yes24.com/Product/Goods/134331993', 'https://www.yes24.com/Product/Goods/128266166', 'https://www.yes24.com/Product/Goods/124999476', 'https://www.yes24.com/Product/Goods/124539805', 'https://www.yes24.com/Product/Goods/8759796', 'https://www.yes24.com/Product/Goods/129602366', 'https://www.yes24.com/Product/Goods/132565140',

In [23]:
len(link_list)

72

In [41]:
#for link in link_list :
    #상세 페이지로 이동


browser.get(link_list[0])

title = browser.find_element(By.CLASS_NAME, 'gd_name').text
author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
rating = browser.find_element(By.CLASS_NAME, "yes_b").text
reviews = browser.find_element(By.CLASS_NAME, "gd_reviewCount").text
sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
price = browser.find_element(By.CLASS_NAME, "yes_m").text[:-1]
ranking = browser.find_element(By.CLASS_NAME, "gd_best").text.split(" | ")[0]
ranking_weeks = browser.find_element(By.CLASS_NAME, "gd_best").text.split(" | ")[1]


'국내도서 1위 2주'